In [1]:
import ortools

load d:\miniconda3\envs\aps-mtrl-analysis\lib\site-packages\ortools\.libs\zlib1.dll...
load d:\miniconda3\envs\aps-mtrl-analysis\lib\site-packages\ortools\.libs\abseil_dll.dll...
load d:\miniconda3\envs\aps-mtrl-analysis\lib\site-packages\ortools\.libs\utf8_validity.dll...
load d:\miniconda3\envs\aps-mtrl-analysis\lib\site-packages\ortools\.libs\re2.dll...
load d:\miniconda3\envs\aps-mtrl-analysis\lib\site-packages\ortools\.libs\libprotobuf.dll...
load d:\miniconda3\envs\aps-mtrl-analysis\lib\site-packages\ortools\.libs\highs.dll...
load d:\miniconda3\envs\aps-mtrl-analysis\lib\site-packages\ortools\.libs\ortools.dll...


In [2]:
import pandas as pd

In [44]:
%%html
<img src="问题描述.png" >

In [49]:
index = [ele for ele in 'ABCDEF']
columns = index.copy()
line_cnt_df = pd.DataFrame(index=index, columns=columns)
line_cnt_df.fillna(0, inplace=True)
line_cnt_df.loc['E','D'] = 3
line_cnt_df.loc['B','C'] = 2
line_cnt_df.loc['A','F'] = 1
line_cnt_df.loc['D','B'] = 1
display(line_cnt_df)

C:\Users\wangc\AppData\Local\Temp\ipykernel_22420\2138812539.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  line_cnt_df.fillna(0, inplace=True)


,A,B,C,D,E,F
A,0,0,0,0,0,1
B,0,0,2,0,0,0
C,0,0,0,0,0,0
D,0,1,0,0,0,0
E,0,0,0,3,0,0
F,0,0,0,0,0,0


In [50]:
data = """0,1,2,14,7,7
1,0,3,13,8,8
2,3,0,15,5,5
14,13,15,0,17,20
7,8,5,17,0,3
7,8,5,20,3,0"""
rslt = []
for line in data.splitlines():
    rslt.append([int(ele) for ele in line.strip().split(',')])
index = [ele for ele in 'ABCDEF']
columns = index.copy()
cost_df = pd.DataFrame(rslt, index=index, columns=columns)
display(cost_df)

,A,B,C,D,E,F
A,0,1,2,14,7,7
B,1,0,3,13,8,8
C,2,3,0,15,5,5
D,14,13,15,0,17,20
E,7,8,5,17,0,3
F,7,8,5,20,3,0


## 问题分解
1. 状态为执航中船只数量

执航船只数量是固定的，为对应$$ \sum_{i}^{e} (装船+卸船+航时_i)*航次_i , e= \{(E,D),(B,C), (A,F),(D,B)\}$$
2. 状态为调度中船只数量

优化港口之间的调度路线

In [51]:
num_status = sum(line_cnt_df.values.flatten())

In [52]:
import numpy as np

In [53]:
np.bool(line_cnt_df.values) *2

array([[0, 0, 0, 0, 0, 2],
       [0, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [54]:
n_service = np.sum(((line_cnt_df  * cost_df) +  (line_cnt_df*2)).values.flatten())
print(n_service)

91


## 调度部分

### 各航线地的补充缺口

In [55]:
(-line_cnt_df + line_cnt_df.T).sum(axis=1)

A   -1
B   -1
C    2
D    2
E   -3
F    1
dtype: int64

### 问题转化成如何调度才能使得每个港口的出入港口数相等，使得总天数耗用最少

In [ ]:
from ortools.linear_solver import pywraplp


In [103]:
solver = pywraplp.Solver.CreateSolver('SCIP')
# Create the variables， cols -> index
vars = pd.DataFrame(index=index, columns=columns)
for i in index:
    for j in index:
        if i != j:
            vars.loc[i,j] = solver.IntVar(0, 100, f'{i}_{j}')
        else:
            vars.loc[i,j] =  solver.IntVar(0, 0, f'{i}_{j}')

In [104]:
solver.Add(solver.Sum(vars.loc['A',:]) >= 1)
solver.Add(solver.Sum(vars.loc['B',:]) >= 1)
solver.Add(solver.Sum(vars.loc['E',:]) >= 3)

solver.Add(solver.Sum(vars.loc[:,'A']) <= 0)
solver.Add(solver.Sum(vars.loc[:,'B']) <= 0)
solver.Add(solver.Sum(vars.loc[:,'C']) <= 2)
solver.Add(solver.Sum(vars.loc[:, 'D']) <= 2)
solver.Add(solver.Sum(vars.loc[:,'E']) <= 0)
solver.Add(solver.Sum(vars.loc[:, 'F']) <= 1)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001E1215BA810> >

In [105]:
# 最小化目标函数
solver.Minimize(solver.Sum(vars.loc[i,j] * cost_df.loc[i,j] for i in index for j in index if i != j))

In [106]:
solver.Solve()

0

### 目标值为在调配途中的船只数量

In [112]:
n_schedule = solver.Objective().Value()
display(n_schedule)

40.0

In [110]:
vars.apply(lambda x: x.apply(lambda y: y.solution_value()))

,A,B,C,D,E,F
A,0.0,0.0,0.0,1.0,0.0,0.0
B,0.0,0.0,0.0,1.0,0.0,0.0
C,0.0,0.0,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,2.0,0.0,0.0,1.0
F,0.0,0.0,0.0,0.0,0.0,0.0


## 总共船只数量

In [116]:
total = n_service + n_schedule
display(f"Total cost: {int(total)}")

'Total cost: 131'